In [2]:
from flask import Flask, render_template, request, redirect, url_for
import re
from urllib.parse import urlparse
from atproto import Client, models
from datetime import datetime
from dateutil import parser
import random

In [27]:

def extract_post_info(url):
    """Extraire les informations d'un post à partir de l'URL Bluesky"""
    # Format attendu: https://bsky.app/profile/[handle]/post/[id]
    pattern = r'bsky\.app/profile/([^/]+)/post/([a-zA-Z0-9]+)'
    match = re.search(pattern, url)
    if match:
        return match.group(1), match.group(2)
    return None, None

def get_post_data(handle, post_id):
    """Récupérer les données d'un post Bluesky"""
    try:
        # Initialiser le client
        client = Client()
        client.login(login='aro402@hotmail.fr', password='gamertag')
        
        # Construire l'URI du post
        uri = f'at://{handle}/app.bsky.feed.post/{post_id}'
        
        # Récupérer le post
        response = client.app.bsky.feed.get_post_thread({'uri': uri})
        thread_data = response.model_dump()
        
        if not thread_data or 'thread' not in thread_data or 'post' not in thread_data['thread']:
            return None
            
        post = thread_data['thread']['post']
        
        # Extraire les images si présentes
        images = []
        try:
            if 'embed' in post and post['embed'] and 'images' in post['embed']:
                for img in post['embed']['images']:
                    images.append({
                        'alt': img.get('alt', ''),
                        'url': img.get('fullsize', img.get('thumb', None))
                    })
        except Exception as e:
            print(f"Erreur lors de l'extraction des images: {str(e)}")
            pass
        
        
        ###### INSERT ANALYZE MODEL HERE ######
        # Générer des scores aléatoires 
        fake_news_prob = random.uniform(0, 100)
        sentiment = random.uniform(-1, 1)
        controversy = random.uniform(0, 1)
        reliability = random.uniform(0, 100)
        
        return {
            'author': {
                'handle': post['author']['handle'],
                'display_name': post['author'].get('displayName', post['author']['handle']),
                'avatar': post['author'].get('avatar', None),
            },
            'content': post['record']['text'],
            'created_at': parser.parse(post['indexed_at']).strftime("%d/%m/%Y %H:%M"),
            'images': images,
            
            ###### INSERT RESULT OF THE ANALYZE MODEL HERE ######
            'metrics': {
                'fake_news_probability': round(fake_news_prob, 1),
                'sentiment_score': round(sentiment, 2),
                'controversy_index': round(controversy, 1),
                'reliability_score': round(reliability, 1)
            },
            'likes': post['like_count'],
            'reposts': post['repost_count'],
            'timestamp': parser.parse(post['indexed_at']).strftime("%d/%m/%Y %H:%M")
        }
    except Exception as e:
        print(f"Erreur lors de la récupération du post: {str(e)}")
        return None

In [28]:
post_infos = extract_post_info("https://bsky.app/profile/profgalloway.com/post/3lhruo7rmo22x")

#  'like_count': 26327,
#  'quote_count': 135,
#  'reply_count': 382,
#  'repost_count': 4159,

In [31]:
handle, post_id = extract_post_info("https://bsky.app/profile/profgalloway.com/post/3lhruo7rmo22x")

client = Client()
client.login(login='aro402@hotmail.fr', password='gamertag')

response = client.app.bsky.feed.get_post_thread({'uri': f'at://{handle}/app.bsky.feed.post/{post_id}'})
thread_data = response.model_dump()

In [40]:
thread_data['thread']['post']['author']

{'did': 'did:plc:e5bl34y4m4zap3g6zg2wxpze',
 'handle': 'profgalloway.com',
 'associated': None,
 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:e5bl34y4m4zap3g6zg2wxpze/bafkreidrdwgbfb4x7am6onokdoe3grude7nsv6tynwezk2tiak6esysr7a@jpeg',
 'created_at': '2024-11-12T16:06:24.236Z',
 'display_name': 'Scott Galloway',
 'labels': [],
 'viewer': {'blocked_by': False,
  'blocking': None,
  'blocking_by_list': None,
  'followed_by': None,
  'following': None,
  'known_followers': None,
  'muted': False,
  'muted_by_list': None,
  'py_type': 'app.bsky.actor.defs#viewerState'},
 'py_type': 'app.bsky.actor.defs#profileViewBasic'}

In [29]:
get_post_data(post_infos[0], post_infos[1])

{'author': {'handle': 'profgalloway.com',
  'display_name': 'profgalloway.com',
  'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:e5bl34y4m4zap3g6zg2wxpze/bafkreidrdwgbfb4x7am6onokdoe3grude7nsv6tynwezk2tiak6esysr7a@jpeg'},
 'content': 'This energy.',
 'created_at': '10/02/2025 00:19',
 'images': [{'alt': '',
   'url': 'https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:e5bl34y4m4zap3g6zg2wxpze/bafkreiecnjihjvowqya4smvqxsbar5pc4qgdkwoin4rkquo7xbr67p3sca@jpeg'}],
 'metrics': {'fake_news_probability': 59.4,
  'sentiment_score': -0.92,
  'controversy_index': 0.5,
  'reliability_score': 50.6},
 'likes': 30318,
 'reposts': 4776,
 'timestamp': '10/02/2025 00:19'}

In [9]:
test = get_post_data2("at://matchingmole.bsky.social/app.bsky.feed.post/3lgvtfrvfg22w")

données post recup
modeldump ok
reading json o k
image k
{'author': {'did': 'did:plc:f7zq4oaiwboqioutbioas6wm', 'handle': 'matchingmole.bsky.social', 'associated': {'chat': {'allow_incoming': 'following', 'py_type': 'app.bsky.actor.defs#profileAssociatedChat'}, 'feedgens': None, 'labeler': None, 'lists': None, 'starter_packs': None, 'py_type': 'app.bsky.actor.defs#profileAssociated'}, 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:f7zq4oaiwboqioutbioas6wm/bafkreihligp3cwojb4sfoeam5e5rccq6cnh7b2lvgvr7kghvg2o2tng77m@jpeg', 'created_at': '2024-08-10T11:32:11.607Z', 'display_name': 'Bernard Alcer dit Balcer', 'labels': [], 'viewer': {'blocked_by': False, 'blocking': None, 'blocking_by_list': None, 'followed_by': None, 'following': None, 'known_followers': None, 'muted': False, 'muted_by_list': None, 'py_type': 'app.bsky.actor.defs#viewerState'}, 'py_type': 'app.bsky.actor.defs#profileViewBasic'}, 'cid': 'bafyreigov22jjgfs2t5nbopvifot6zr7n3olphsro2xce77wd6oicijr3a', 'indexed_at': 

In [10]:
test

{'author': {'handle': 'matchingmole.bsky.social',
  'display_name': 'Bernard Alcer dit Balcer',
  'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:f7zq4oaiwboqioutbioas6wm/bafkreihligp3cwojb4sfoeam5e5rccq6cnh7b2lvgvr7kghvg2o2tng77m@jpeg'},
 'content': '«\xa0Un prof\xa0» n’est pas sexué.',
 'timestamp': '2025-01-29 20:42:02',
 'likes': 0,
 'reposts': 0,
 'images': [],
 'metrics': {'fake_news_probability': 49,
  'sentiment_score': -0.28,
  'controversy_index': 0.57,
  'reliability_score': 62}}

## ACCOUNT INFO 

In [50]:
handle, post_id = extract_post_info("https://bsky.app/profile/profgalloway.com/post/3lhruo7rmo22x")

client = Client()
client.login(login='aro402@hotmail.fr', password='gamertag')

response = client.app.bsky.feed.get_post_thread({'uri': f'at://{handle}/app.bsky.feed.post/{post_id}'})
thread_data = response.model_dump()

did = thread_data['thread']['post']['author']['did']

account_info = client.app.bsky.actor.get_profile({'actor' : did}).model_dump()


account_info




{'did': 'did:plc:e5bl34y4m4zap3g6zg2wxpze',
 'handle': 'profgalloway.com',
 'associated': {'chat': None,
  'feedgens': 0,
  'labeler': False,
  'lists': 0,
  'starter_packs': 0,
  'py_type': 'app.bsky.actor.defs#profileAssociated'},
 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:e5bl34y4m4zap3g6zg2wxpze/bafkreidrdwgbfb4x7am6onokdoe3grude7nsv6tynwezk2tiak6esysr7a@jpeg',
 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:e5bl34y4m4zap3g6zg2wxpze/bafkreiburs3mnwfl5i4no5zwu3ixexax54355ea5n73swpkm27jcy2anl4@jpeg',
 'created_at': '2024-11-12T16:06:24.236Z',
 'description': 'Professor of Marketing at NYU Stern School of Business, serial entrepreneur, and host of the Prof G and Pivot Podcasts.',
 'display_name': 'Scott Galloway',
 'followers_count': 121335,
 'follows_count': 60,
 'indexed_at': '2024-11-12T16:12:50.640Z',
 'joined_via_starter_pack': None,
 'labels': [],
 'pinned_post': None,
 'posts_count': 245,
 'viewer': {'blocked_by': False,
  'blocking': None,
  'blocking_

In [51]:
list(account_info)

'avatar',
'created_at',
'description',
'display_name',
'followers_count',
'follows_count',
'posts_count'

['did',
 'handle',
 'associated',
 'avatar',
 'banner',
 'created_at',
 'description',
 'display_name',
 'followers_count',
 'follows_count',
 'indexed_at',
 'joined_via_starter_pack',
 'labels',
 'pinned_post',
 'posts_count',
 'viewer',
 'py_type']

# Feed_test 

In [5]:
def get_feed_data(handle, feed_id):
    """Récupérer les données d'un feed Bluesky"""
    try:
        # Initialiser le client
        client = Client()
        client.login(login='aro402@hotmail.fr', password='gamertag')
        
        # Récupérer le feed
        feed_view = client.get_feed_generator(actor="trending.bsky.app", feed="60160961")
        if not feed_view:
            return None

        # Récupérer les 10 derniers posts du feed
        feed = client.get_feed(feed=f'at://{handle}/app.bsky.feed.generator/{feed_id}', limit=10)
        if not feed or not feed.feed:
            return None

        posts_data = []
        total_fake_news_prob = 0
        total_sentiment = 0
        total_controversy = 0
        total_reliability = 0

        for post in feed.feed:
            # Générer des scores aléatoires pour chaque post
            fake_news_prob = random.uniform(0, 100)
            sentiment = random.uniform(-1, 1)
            controversy = random.uniform(0, 100)
            reliability = random.uniform(0, 100)

            # Accumuler pour les moyennes
            total_fake_news_prob += fake_news_prob
            total_sentiment += sentiment
            total_controversy += controversy
            total_reliability += reliability

            # Extraire les images si présentes
            images = []
            try:
                if post.post.embed and hasattr(post.post.embed, 'images'):
                    for img in post.post.embed.images:
                        images.append({
                            'alt': img.alt,
                            'url': img.fullsize
                        })
            except:
                pass

            # Formater les données du post
            post_data = {
                'author': {
                    'handle': post.post.author.handle,
                    'display_name': post.post.author.display_name,
                    'avatar': post.post.author.avatar,
                },
                'content': post.post.record.text,
                'created_at': parser.parse(post.post.record.created_at).strftime("%d/%m/%Y %H:%M"),
                'images': images,
                'stats': {
                    'fake_news_probability': round(fake_news_prob, 1),
                    'sentiment_score': round(sentiment, 2),
                    'controversy_index': round(controversy, 1),
                    'reliability_score': round(reliability, 1)
                }
            }
            posts_data.append(post_data)

        # Calculer les moyennes pour le feed entier
        num_posts = len(posts_data)
        feed_stats = {
            'fake_news_probability': round(total_fake_news_prob / num_posts, 1),
            'sentiment_score': round(total_sentiment / num_posts, 2),
            'controversy_index': round(total_controversy / num_posts, 1),
            'reliability_score': round(total_reliability / num_posts, 1)
        }

        return {
            'feed_info': {
                'name': feed_view.view.display_name,
                'description': feed_view.view.description,
                'avatar': feed_view.view.avatar,
            },
            'stats': feed_stats,
            'posts': posts_data
        }

    except Exception as e:
        print(f"Erreur lors de la récupération du feed: {str(e)}")
        return None

In [13]:
client.app.bsky.actor.get_profile({'actor' : 'bsky.app'}).model_dump()['did']

'did:plc:z72i7hdynmk6r22z27h6tvur'

In [23]:
url = 'https://bsky.app/profile/trending.bsky.app/feed/60893389'
pattern = r'https://bsky\.app/profile/([^/]+)/feed/([^/]+)'
match = re.match(pattern, url)
actor, feed_id = match.groups()


did = client.app.bsky.actor.get_profile({'actor' : actor}).model_dump()['did']
feed = client.app.bsky.feed.get_feed({"feed" : f"at://{did}/app.bsky.feed.generator/{feed_id}"})

feed_data = feed.model_dump()

In [42]:
for item in feed_data['feed']:
        print(item['post']['indexed_at'])

2025-01-30T19:57:28.825Z
2025-01-30T19:55:54.222Z
2025-01-30T20:02:17.630Z
2025-01-30T20:02:02.734Z
2025-01-30T16:46:35.033Z
2025-01-30T18:43:26.225Z
2025-01-30T17:39:58.320Z
2025-01-30T15:50:46.126Z
2025-01-30T17:50:16.429Z
2025-01-30T18:17:20.430Z
2025-01-30T15:17:49.721Z
2025-01-30T15:17:49.721Z
2025-01-30T18:38:55.520Z
2025-01-30T15:42:08.521Z
2025-01-30T18:11:57.219Z
2025-01-30T16:09:56.217Z
2025-01-30T17:06:48.542Z
2025-01-30T16:12:45.516Z
2025-01-30T17:10:47.628Z
2025-01-30T18:28:51.423Z
2025-01-30T17:44:42.924Z
2025-01-30T19:35:39.633Z
2025-01-30T19:20:40.516Z
2025-01-30T16:57:09.732Z
2025-01-30T15:34:24.520Z


In [44]:
for item in feed_data['feed']:
        post = item['post']
        print(f"Traitement du post: {post['record']['text'][:100]}...")
        
        # Générer des scores aléatoires pour chaque post
        fake_news_prob = random.uniform(0, 100)
        sentiment = random.uniform(-1, 1)
        controversy = random.uniform(0, 100)
        reliability = random.uniform(0, 100)

        # Accumuler pour les moyennes
        # total_fake_news_prob += fake_news_prob
        # total_sentiment += sentiment
        # total_controversy += controversy
        # total_reliability += reliability

        # Extraire les images si présentes
        images = []
        try:
            if 'embed' in post and post['embed'] and 'images' in post['embed']:
                for img in post['embed']['images']:
                    images.append({
                        'alt': img.get('alt', ''),
                        'url': img.get('fullsize', img.get('thumb', None))
                    })
        except Exception as e:
            print(f"Erreur lors de l'extraction des images: {str(e)}")
            pass


        # Formater les données du post
        post_data = {
            'author': {
                'handle': post['author']['handle'],
                'display_name': post['author'].get('displayName', post['author']['handle']),
                'avatar': post['author'].get('avatar', None),
            },
            'content': post['record']['text'],
            'created_at': parser.parse(post['indexed_at']).strftime("%d/%m/%Y %H:%M")
            # 'images': images,
            # 'stats': {
            #     'fake_news_probability': round(fake_news_prob, 1),
            #     'sentiment_score': round(sentiment, 2),
            #     'controversy_index': round(controversy, 1),
            #     'reliability_score': round(reliability, 1)
            # }
        }

        print(post_data)
        # posts_data.append(post_data)

        print('FORMAT DATA OK ')

Traitement du post: Oh snap!!!

Senator Cory Booker points out that he hears Elon Musk is trying to place people to work...
{'author': {'handle': 'artcandee.bsky.social', 'display_name': 'artcandee.bsky.social', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:pmhyscv7t7incelmoevkijpr/bafkreic6uhshdddxkyyplrdvzoph2j2vzxw4h3jlycdvfbsti5bv6pknia@jpeg'}, 'content': 'Oh snap!!!\n\nSenator Cory Booker points out that he hears Elon Musk is trying to place people to work with Kash Patel.\n\n🍿', 'created_at': '30/01/2025 19:57'}
FORMAT DATA OK 
Traitement du post: Anyone ask Kash Patel about being on the Board of Truth Social or being one of Trump’s NARA reps whe...
{'author': {'handle': 'out5p0ken.bsky.social', 'display_name': 'out5p0ken.bsky.social', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:eezrj7dplg2goacopghccdgq/bafkreiailqhk7c5wfie75yc3e5wmxi4h2cldglqwzw6cja7rrn42iesn6q@jpeg'}, 'content': 'Anyone ask Kash Patel about being on the Board of Truth Social or being one

# GET POST with tags



In [2]:
client = Client()
client.login(login='aro402@hotmail.fr', password='gamertag')

searchresult = client.app.bsky.feed.search_posts({'q' : "trump",
                                                'sort' : 'top'})



In [20]:
def get_tag_datas(user_tag) :
    ''' Récupérer les informations liés à un tag'''
    try:
        client = Client()
        client.login(login='aro402@hotmail.fr', password='gamertag')

        searchresult = client.app.bsky.feed.search_posts({'q' : user_tag,
                                                        'sort' : 'top'})

        if searchresult.model_dump()['posts'] == [] : 
            
            return 'SPOTTED'
            


        posts_data = []
        total_fake_news_prob = 0
        total_sentiment = 0
        total_controversy = 0
        total_reliability = 0

        for item in searchresult.model_dump()['posts']:
            post = item
            # print(f"Traitement du post: {post['record']['text'][:100]}...")
            
            # Générer des scores aléatoires pour chaque post
            fake_news_prob = random.uniform(0, 100)
            sentiment = random.uniform(-1, 1)
            controversy = random.uniform(0, 100)
            reliability = random.uniform(0, 100)

            # Accumuler pour les moyennes
            total_fake_news_prob += fake_news_prob
            total_sentiment += sentiment
            total_controversy += controversy
            total_reliability += reliability

            # Extraire les images si présentes
            images = []
            try:
                if 'embed' in post and post['embed'] and 'images' in post['embed']:
                    for img in post['embed']['images']:
                        images.append({
                            'alt': img.get('alt', ''),
                            'url': img.get('fullsize', img.get('thumb', None))
                        })
            except Exception as e:
                print(f"Erreur lors de l'extraction des images: {str(e)}")
                pass

            # Formater les données du post
            post_data = {
                'author': {
                    'handle': post['author']['handle'],
                    'display_name': post['author'].get('displayName', post['author']['handle']),
                    'avatar': post['author'].get('avatar', None),
                },
                'content': post['record']['text'],
                'created_at': parser.parse(post['indexed_at']).strftime("%d/%m/%Y %H:%M"),
                'images': images,
                'stats': {
                    'fake_news_probability': round(fake_news_prob, 1),
                    'sentiment_score': round(sentiment, 2),
                    'controversy_index': round(controversy, 1),
                    'reliability_score': round(reliability, 1)
                }
            }
            posts_data.append(post_data)

        # Calculer les moyennes pour le feed entier
        num_posts = len(posts_data)
        allposts_tags_stats = {
            'fake_news_probability': round(total_fake_news_prob / num_posts, 1),
            'sentiment_score': round(total_sentiment / num_posts, 2),
            'controversy_index': round(total_controversy / num_posts, 1),
            'reliability_score': round(total_reliability / num_posts, 1)
        }
        
        return {
            'tag_info': {
                'name': user_tag,
                'description': user_tag,
                'avatar': user_tag
            },
            'stats': allposts_tags_stats,
            'posts': posts_data
        }
        
    except Exception as e:
        
        print(f"Erreur lors de l'extraction des images: {str(e)}")
        pass

In [22]:
test = get_tag_datas('trump')


In [25]:
test['posts']


[{'author': {'handle': 'theonion.com',
   'display_name': 'theonion.com',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:a4pqq234yw7fqbddawjo7y35/bafkreiajdz4ktkjv5jrnye7sotgz6g5xarts6yu6ypbhninxqnvwoiwsfi@jpeg'},
  'content': 'Trump Asks Which One The Ball theonion.com/trump-a... ',
  'created_at': '10/02/2025 00:22',
  'images': [{'alt': '',
    'url': 'https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:a4pqq234yw7fqbddawjo7y35/bafkreigvfwtjylrznppb4lul2cilmuxh7ghrbtk4w7up3k4lojp4wx6a7i@jpeg'}],
  'stats': {'fake_news_probability': 87.8,
   'sentiment_score': -0.97,
   'controversy_index': 1.4,
   'reliability_score': 52.1}},
 {'author': {'handle': 'mjfree.bsky.social',
   'display_name': 'mjfree.bsky.social',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gkgmduxh722ocstroyi75gbg/bafkreifbzf5kuau7gomna24zifgthxvsgjyvikvpv4chktct5qcgbtdjsy@jpeg'},
  'content': 'BOO DONALD TRUMP AT THE SUPER BOWL\nBOO DONALD TRUMP AT THE SUPER BOWL\nBOO DONALD TRUMP AT THE 

In [54]:
for post in searchresult.model_dump()['posts'] : 
    
    print(post['record'])

{'created_at': '2025-02-10T00:22:42.000Z', 'text': 'Trump Asks Which One The Ball theonion.com/trump-a... ', 'embed': {'images': [{'alt': '', 'image': {'mime_type': 'image/jpeg', 'size': 87604, 'ref': {'link': 'bafkreigvfwtjylrznppb4lul2cilmuxh7ghrbtk4w7up3k4lojp4wx6a7i'}, 'py_type': 'blob'}, 'aspect_ratio': {'height': 1080, 'width': 864, 'py_type': 'app.bsky.embed.defs#aspectRatio'}, 'py_type': 'app.bsky.embed.images#image'}], 'py_type': 'app.bsky.embed.images'}, 'entities': None, 'facets': [{'features': [{'uri': 'https://theonion.com/trump-asks-which-one-the-ball/', 'py_type': 'app.bsky.richtext.facet#link'}], 'index': {'byte_end': 53, 'byte_start': 30, 'py_type': 'app.bsky.richtext.facet#byteSlice'}, 'py_type': 'app.bsky.richtext.facet'}], 'labels': None, 'langs': [], 'reply': None, 'tags': None, 'py_type': 'app.bsky.feed.post'}
{'created_at': '2025-02-07T19:17:46.823Z', 'text': 'BOO DONALD TRUMP AT THE SUPER BOWL\nBOO DONALD TRUMP AT THE SUPER BOWL\nBOO DONALD TRUMP AT THE SUPER BO

In [33]:

def extract_post_info(url):
    """Extraire les informations d'un post à partir de l'URL Bluesky"""
    # Format attendu: https://bsky.app/profile/[handle]/post/[id]
    pattern = r'bsky\.app/profile/([^/]+)/post/([a-zA-Z0-9]+)'
    match = re.search(pattern, url)
    if match:
        return match.group(1), match.group(2)
    return None, None

def get_post_data(handle, post_id):
    """Récupérer les données d'un post Bluesky"""
    try:
        # Initialiser le client
        client = Client()
        client.login(login='aro402@hotmail.fr', password='gamertag')
        
        # Construire l'URI du post
        uri = f'at://{handle}/app.bsky.feed.post/{post_id}'
        
        # Récupérer le post
        response = client.app.bsky.feed.get_post_thread({'uri': uri})
        thread_data = response.model_dump()
        
        if not thread_data or 'thread' not in thread_data or 'post' not in thread_data['thread']:
            return None
            
        post = thread_data['thread']['post']
        
        # Extraire les images si présentes
        images = []
        try:
            if 'embed' in post and post['embed'] and 'images' in post['embed']:
                for img in post['embed']['images']:
                    images.append({
                        'alt': img.get('alt', ''),
                        'url': img.get('fullsize', img.get('thumb', None))
                    })
        except Exception as e:
            print(f"Erreur lors de l'extraction des images: {str(e)}")
            pass
        
        
        ###### INSERT ANALYZE MODEL HERE ######
        # Générer des scores aléatoires 
        fake_news_prob = random.uniform(0, 100)
        sentiment = random.uniform(-1, 1)
        controversy = random.uniform(0, 1)
        reliability = random.uniform(0, 100)
        
        return {
            'author': {
                'handle': post['author']['handle'],
                'display_name': post['author'].get('displayName', post['author']['handle']),
                'avatar': post['author'].get('avatar', None),
            },
            'content': post['record']['text'],
            'created_at': parser.parse(post['indexed_at']).strftime("%d/%m/%Y %H:%M"),
            'images': images,
            
            ###### INSERT RESULT OF THE ANALYZE MODEL HERE ######
            'metrics': {
                'fake_news_probability': round(fake_news_prob, 1),
                'sentiment_score': round(sentiment, 2),
                'controversy_index': round(controversy, 1),
                'reliability_score': round(reliability, 1)
            },
            'likes': post['like_count'],
            'reposts': post['repost_count'],
            'timestamp': parser.parse(post['indexed_at']).strftime("%d/%m/%Y %H:%M")
        }
    except Exception as e:
        print(f"Erreur lors de la récupération du post: {str(e)}")
        return None